In [45]:
import matplotlib.pyplot as plt
import numpy as np
import random

## Classes & Functions

#### Algorithms

In [191]:
class SimulatedAnnealing:
    def __init__(self, starting_route, temperature, rate, nodes, markov_length, temperature_minimum=1e-5):
        self.initial_solution = starting_route
        self.current_solution = self.initial_solution
        self.temperature = temperature
        self.nodes = nodes
        self.markov_length = markov_length
        self.temperature_minimum = temperature_minimum
        self.n = len(self.initial_solution)
        self.rate = rate

        self.current_score = self.distance(self.initial_solution)

    def distance(self, current_state) -> int:
        return sum([self.nodes[current_state[i]].distance(self.nodes[current_state[i + 1]]) for i in range(len(current_state) - 1)])

    def update_temperature(self) -> None:
        self.temperature = self.temperature * self.rate

    def two_opt_move(self, solution) -> list:
        i, j = np.sort(np.random.randint(1, self.n - 1, 2))
        new_solution = solution[:i] + solution[i:j][::-1] + solution[j:]
        return new_solution

    def run(self) -> list:
        while self.temperature > self.temperature_minimum:
            for _ in range(self.markov_length):
                cur_sol = self.current_solution
                new_solution = self.two_opt_move(cur_sol)
                new_score = self.distance(new_solution)

                if self.current_score > new_score:
                    self.current_score = new_score
                    self.current_solution = new_solution
                else:
                    score_diff = self.current_score - new_score
                    if np.exp(score_diff/self.temperature) > random.random():
                        self.current_solution = new_solution
                        self.current_score = new_score

            self.update_temperature()

        return self.current_solution, self.current_score

In [94]:
def create_random(starting_list) -> list:
    """
    Creates random initial solution, it shuffles the list and returns it.
    """

    random.shuffle(starting_list)
    starting_list.append(1)
    return starting_list

#### Network

In [31]:
class Node:
    def __init__(self, coordinates, number):
        self.number = number
        self.x, self.y = coordinates

    def distance(self, b) -> int:
        return np.linalg.norm(np.array(self.get_coordinates())
            - np.array(b.get_coordinates()))

    def get_coordinates(self) -> list:
        return [self.x, self.y]

#### Helper

In [68]:
def load_data(file_name) -> dict:
    """
    Loads in a text file with the given file name and creates a node for every
    point. Returns a list of nodes.
    """
    file = open(f'TSP-Configurations/{file_name}.tsp.txt', "r")
    file = file.read().splitlines()[6:-1]
    nodes = dict()
    ids = list()

    # Go through every line of the file and create node for every line.
    for node in file:
        values = [int(value) for value in node.split()]
        nodes[values[0]] = Node([values[1], values[2]], values[0])
        ids.append(values[0])

    # Return the list of nodes.
    return nodes, ids


In [56]:
def calculate_optimal(nodes, file_name) -> float:
    """
    Takes the dictionary of nodes and calculates the distance travelled of the
    optimal tour. This serves as a benchmark for the Simulated Annealing
    algorithm.
    """

    optimal = open(f'TSP-Configurations/{file_name}.opt.tour.txt', "r")
    optimal = optimal.read().splitlines()[5:-1]
    return sum([nodes[int(optimal[i])].distance(nodes[abs(int(optimal[i + 1]))]) for i in range(len(optimal) - 1)])

In [88]:
def calculate_current_score(nodes, current_state) -> float:
    return sum([nodes[current_state[i]].distance(nodes[current_state[i + 1]]) for i in range(len(current_state) - 1)])


## Testing

Testing happens on the configuration of the eli51.tsp file. For experiments the bigger
network a280.tsp is used. 

In [195]:
# Create simple TSP
file_name = 'eil51'
nodes, initial_list = load_data(file_name)
copy_initial = initial_list.copy()
temperature = 10000
rate = 0.9999
markov_length = 10

In [196]:
random_start = create_random(copy_initial)
siman = SimulatedAnnealing(random_start, temperature, rate, nodes, markov_length)

In [197]:
solution, score = siman.run()

In [198]:
print(score)

470.8302769307382


In [178]:
calculate_current_score(nodes, initial_list)

1299.575900454896

In [177]:
random_start = create_random(copy_initial)


In [199]:
calculate_current_score(nodes, random_start)

1644.0155205088752

In [180]:
calculate_optimal(nodes, file_name)

429.98331198338406

## Experiments